# 功能描述
- 遍历目录下所有网关LOG文件，锁定所有mc close到mc open间60/61传感器在5s内同时报有人的情况
- 去除同一时间点的多次上报
- 仅关注6月23日以后

In [1]:
from datetime import datetime
import os
from os import path

In [2]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [3]:
time_start_point = datetime(2022, 6, 23, 0, 0, 0, 0)

mc_open_string = "mc open"
mc_close_string = "mc close"
gateway_powerup_string = "R2000 init ok"

bedroom_target_string = "485 induc,id:60,sta:1"
bathroom_target_string = "485 induc,id:61,sta:1"
ir_string = "485 induc,id:6a"

In [15]:
def check_mc_close_points(file_name):
    result_list = []
    global time_start_point
    global temp_result1_date_time_obj
    global temp_result2_date_time_obj
    global temp_result3_date_time_obj
    
    with open(file_name, 'r') as f:
        txt_lines = f.readlines()
            
    for i in range(0, len(txt_lines)):
        if txt_lines[i].find(mc_close_string) >= 0:
            pos1 = txt_lines[i].find('[') + 1
            pos2 = txt_lines[i].find(']')
            date_time_str = txt_lines[i][pos1:pos2].strip()
            date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d-%H-%M-%S')
            was_after_data = time_start_point < date_time_obj and date_time_obj.hour > 18
            if was_after_data:
                result_list.append(i+1)
    return result_list

In [16]:
def check_mc_open_points(file_name):
    result_list = []
    global time_start_point
    global temp_result1_date_time_obj
    global temp_result2_date_time_obj
    global temp_result3_date_time_obj
    
    with open(file_name, 'r') as f:
        txt_lines = f.readlines()
            
    for i in range(0, len(txt_lines)):
        if txt_lines[i].find(mc_open_string) >= 0:
            pos1 = txt_lines[i].find('[') + 1
            pos2 = txt_lines[i].find(']')
            date_time_str = txt_lines[i][pos1:pos2].strip()
            date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d-%H-%M-%S')
            was_after_data = time_start_point < date_time_obj
            if was_after_data:
                result_list.append(i+1)
    return result_list

In [17]:
def check_gateway_init_points(file_name):
    result_list = []
    global time_start_point
    global temp_result1_date_time_obj
    global temp_result2_date_time_obj
    global temp_result3_date_time_obj
    
    with open(file_name, 'r') as f:
        txt_lines = f.readlines()
            
    for i in range(0, len(txt_lines)):
        if txt_lines[i].find(gateway_powerup_string) >= 0:
            pos1 = txt_lines[i].find('[') + 1
            pos2 = txt_lines[i].find(']')
            date_time_str = txt_lines[i][pos1:pos2].strip()
            date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d-%H-%M-%S')
            was_after_data = time_start_point < date_time_obj
            if was_after_data:
                result_list.append(i+1)
    return result_list

In [18]:
close_list = check_mc_close_points('./data/201.txt')
open_list = check_mc_open_points('./data/201.txt')
gateway_init = check_gateway_init_points('./data/201.txt')

In [19]:
print(close_list)
print(open_list)
print(gateway_init)

[]
[3677, 3712, 3731, 3756, 3758, 3787, 3886, 3915, 3917, 3919, 3921, 4641]
[]


# 得到成对的close与open
- 并去掉中间有网关重启动的对
- 并去掉中间无间隔的对

In [20]:
close_open_pair = []
for i in range(0, len(close_list)):
    for j in range(0, len(open_list)):
        if open_list[j] > close_list[i]:
            close_open_pair.append((close_list[i], open_list[j]))
            break;
    
for i in range(0, len(gateway_init)):
    for up, down in close_open_pair:
        if gateway_init[i] < down and gateway_init[i] > up:
            #print(f'find {up}(mc_close)-{gateway_init[i]}(R2000 power up)-{down}(mc_open)')
            if (up, down) in close_open_pair:
                close_open_pair.remove((up, down))
        if down == up + 1:
            #print(f'find {up}(mc_close)-{down}(mc_open)')
            if (up, down) in close_open_pair:
                close_open_pair.remove((up, down))
        

In [21]:
print(close_open_pair)

[]


In [22]:
def p60_61_1(file_name, pair):
    bed_error = 0
    bath_error = 0
    error_counter = 0
    with open(file_name, 'r') as f:
        txt_lines = f.readlines()
        
    for up, down in pair:
        for i in range(up, down - 1):
            have_ir = txt_lines[i].find(ir_string) >= 0
            if not have_ir:
                if txt_lines[i].find(bedroom_target_string) >= 0:
                    #print(f'{bcolors.OKBLUE}bed ERROR: {i+1}-{txt_lines[i]}{bcolors.ENDC}')
                    bed_error = 1
                if txt_lines[i].find(bathroom_target_string) >= 0:
                    #print(f'{bcolors.WARNING}bath ERROR: {i+1}-{txt_lines[i]}{bcolors.ENDC}')
                    bath_error = 1
        if bed_error and bath_error:
            print(f'{up}-{down}')
            error_counter = error_counter + 1
        bed_error = 0
        bath_error = 0
    if error_counter > 0:
        print(file_name)
        print('-----------')

In [23]:
p60_61_1('./data/201.txt', close_open_pair)

In [24]:
def scan_files(url):
    file  = os.listdir(url)
    close_list = []
    open_list = []
    gateway_init = []
    close_open_pair = []
    
    for f in file:
        real_url = path.join(url, f)
        if path.isfile(real_url):
            #print(path.abspath(real_url))
            close_list = check_mc_close_points(path.abspath(real_url))
            open_list = check_mc_open_points(path.abspath(real_url))
            gateway_init = check_gateway_init_points(path.abspath(real_url))

            for i in range(0, len(close_list)):
                for j in range(0, len(open_list)):
                    if open_list[j] > close_list[i]:
                        close_open_pair.append((close_list[i], open_list[j]))
                        break;

            for i in range(0, len(gateway_init)):
                for up, down in close_open_pair:
                    if gateway_init[i] < down and gateway_init[i] > up:
                        #print(f'find {up}(mc_close)-{gateway_init[i]}(R2000 power up)-{down}(mc_open)')
                        if (up, down) in close_open_pair:
                            close_open_pair.remove((up, down))
                    if down == up + 1:
                        #print(f'find {up}(mc_close)-{down}(mc_open)')
                        if (up, down) in close_open_pair:
                            close_open_pair.remove((up, down))
            #print(close_open_pair)
            p60_61_1(path.abspath(real_url), close_open_pair)
                
            close_open_pair = []
            # 如果是文件，则以绝度路径的方式输出
        elif path.isdir(real_url):
            #如果是目录，则是递归自定义函数 scaner_file (url)进行多次
            scaner_file(real_url)
        else: 
            print("其他情况")
            pass

In [25]:
scan_files("./data")

7663-8050
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\208.txt
-----------
72-85
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\212.txt
-----------
8845-8893
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\231.txt
-----------
1532-1577
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\335.txt
-----------
2734-5591
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\511.txt
-----------
